# Hierarchical Indexing
## A Multiply Indexed Series
### The bad way

In [1]:
import pandas as pd
import numpy as np

In [2]:
index = [
    ('California', 2000),
    ('California', 2010),
    ('New York', 2000),
    ('New York', 2010),
    ('Texas', 2000),
    ('Texas', 2010)
]
populations = [
    33871648, 37253956,
    18976457, 19378102,
    20851820, 25145561
]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### The better way: Pandas MultiIndex

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as extra dimension

In [8]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [9]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [10]:
pop_df = pd.DataFrame(
    {
        'total': pop,
        'under18': [
            9267089, 9284094,
            4687374, 4318033,
            5906301, 6879014
            ]
    }
)
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Methods of MultiIndex Creation

In [12]:
df = pd.DataFrame(np.random.rand(4, 2),
index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
columns=['data1', 'data2'])
df

data1     data2
a 1  0.528374  0.568176
  2  0.784129  0.232214
b 1  0.153396  0.107888
  2  0.666020  0.814232

In [13]:
data = {
    ('California', 2000): 33871648,
    ('California', 2010): 37253956,
    ('Texas', 2000): 20851820,
    ('Texas', 2010): 25145561,
    ('New York', 2000): 18976457,
    ('New York', 2010): 19378102
}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors

In [14]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [15]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [16]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [20]:
pd.MultiIndex(
    levels=[['a', 'b'], [1, 2]],
    labels=[[0, 0, 1, 1], [0, 1, 0, 1]]
) # labels está descontinuado

TypeError: __new__() got an unexpected keyword argument 'labels'

### MultiIndex level names

In [21]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

In [23]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product(
    [[2013, 2014], [1, 2]],
    names=['year', 'visit']
)
columns = pd.MultiIndex.from_product(
    [['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
    names=['subject', 'type']
)

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      44.0  38.7  29.0  36.9  42.0  37.7
     2      38.0  38.1  41.0  37.3  46.0  36.0
2014 1      46.0  36.2  51.0  35.5  41.0  37.4
     2      25.0  37.6  33.0  39.3  54.0  34.7

In [24]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      29.0  36.9
     2      41.0  37.3
2014 1      51.0  35.5
     2      33.0  39.3

## Indexing and Slicing a MultiIndex
### Multiply indexed Series

In [25]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [26]:
pop['California', 2000]

33871648

In [27]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [28]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [29]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [30]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [31]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

In [32]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      44.0  38.7  29.0  36.9  42.0  37.7
     2      38.0  38.1  41.0  37.3  46.0  36.0
2014 1      46.0  36.2  51.0  35.5  41.0  37.4
     2      25.0  37.6  33.0  39.3  54.0  34.7

In [33]:
health_data['Guido', 'HR']

year  visit
2013  1        29.0
      2        41.0
2014  1        51.0
      2        33.0
Name: (Guido, HR), dtype: float64

In [34]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      44.0  38.7
     2      38.0  38.1

In [35]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        44.0
      2        38.0
2014  1        46.0
      2        25.0
Name: (Bob, HR), dtype: float64

In [36]:
health_data.loc[(:, 1), (:, 'HR')] # incorrecto

SyntaxError: invalid syntax (Temp/ipykernel_8576/3311942670.py, line 1)

In [37]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']] # correcto

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,44.0,29.0,42.0
2014,1,46.0,51.0,41.0


## Rearranging Multi-Indices
### Sorted and unsorted indices

In [38]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.049862
      2      0.337002
c     1      0.894975
      2      0.384686
b     1      0.006479
      2      0.161153
dtype: float64

In [39]:
try:
    data['a':'b'] # si la data no está ordenada nos arroja error
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [40]:
data = data.sort_index()
data

char  int
a     1      0.049862
      2      0.337002
b     1      0.006479
      2      0.161153
c     1      0.894975
      2      0.384686
dtype: float64

In [41]:
data['a':'b']

char  int
a     1      0.049862
      2      0.337002
b     1      0.006479
      2      0.161153
dtype: float64

### Stacking and unstacking indices

In [42]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [43]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [44]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [45]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

In [46]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [47]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data Aggregations on Multi-Indices

In [48]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      44.0  38.7  29.0  36.9  42.0  37.7
     2      38.0  38.1  41.0  37.3  46.0  36.0
2014 1      46.0  36.2  51.0  35.5  41.0  37.4
     2      25.0  37.6  33.0  39.3  54.0  34.7

In [49]:
data_mean = health_data.mean(level='year') # FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
data_mean

C:\Users\EVOO\AppData\Local\Temp/ipykernel_8576/2845460446.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean = health_data.mean(level='year')


subject   Bob       Guido         Sue       
type       HR  Temp    HR  Temp    HR   Temp
year                                        
2013     41.0  38.4  35.0  37.1  44.0  36.85
2014     35.5  36.9  42.0  37.4  47.5  36.05

In [50]:
data_mean.mean(axis=1, level='type')

C:\Users\EVOO\AppData\Local\Temp/ipykernel_8576/1641729190.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean.mean(axis=1, level='type')


type,HR,Temp
year,,
2013,40.000000,37.450000
2014,41.666667,36.783333
